# LLM monitoring with Langchain and AWS Bedrock

This notebook example shows how to integrate Monitoring with Langfuse for langchain and AWS Bedrock.

### Create a multi LLM langchain interface

In [1]:
from langchain_aws.chat_models import BedrockChat
from langchain_core.runnables import ConfigurableField

In [2]:
aws_region_name = "us-east-1"
credentials_profile_name = "perso"
claude_3_sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"
mistral_large = "mistral.mistral-large-2402-v1:0"

In [3]:
mistral_large_bedrock_chat = BedrockChat(
    model_id=mistral_large,
    credentials_profile_name=credentials_profile_name,
    region_name=aws_region_name,
)

/Users/ravindu.somawansa@airliquide.com/Library/Caches/pypoetry/virtualenvs/nteract-Y_KEHMmN-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ChatBedrock instead.
  warn_deprecated(


In [4]:
_model_alternatives = {
    "mistral_large": mistral_large_bedrock_chat
}

In [5]:
claude_3_sonnet = BedrockChat(
    model_id=claude_3_sonnet,
    credentials_profile_name=credentials_profile_name,
    region_name=aws_region_name,
)

In [6]:
bedrock_llm = claude_3_sonnet.configurable_alternatives(
    which=ConfigurableField(
        id="model", name="Model", description="The model that will be used"
    ),
    default_key="claude_3_sonnet",
    **_model_alternatives,
)

### Create a multi LLM langchain prompt

In [7]:
from langchain_core.prompts import PromptTemplate

In [8]:
_MISTRAL_PROMPT = PromptTemplate.from_template(
    """
<s>[INST] You are a conversational AI designed to answer in a friendly way to a question.
You should always answer in rhymes.

Human:
<human_reply>
{input}
</human_reply>

Generate the AI's response.[/INST]</s>
"""
)

In [9]:
_CLAUDE_PROMPT = PromptTemplate.from_template(
    """
You are a conversational AI designed to answer in a friendly way to a question.
You should always answer in jokes.

Human:
<human_reply>
{input}
</human_reply>

Assistant:
"""
)

In [10]:
_CHAT_PROMPT_ALTERNATIVES = {"mistral_large": _MISTRAL_PROMPT}

In [11]:
CONFIGURABLE_CHAT_PROMPT = _CLAUDE_PROMPT.configurable_alternatives(
    which=ConfigurableField(
        id="model",
        name="Model",
        description="The model that will be used",
    ),
    default_key="claude_3_sonnet",
    **_CHAT_PROMPT_ALTERNATIVES
)

### Integrate Langfuse

In [12]:
from langfuse.callback import CallbackHandler
from langchain.schema.runnable import RunnableConfig

langfuse_handler = CallbackHandler(
  secret_key="sk-lf-3821dcfc-4d7d-4e01-be63-b8640b8118b7",
  public_key="pk-lf-e86ffa23-036b-4620-b333-c697d126819a",
  host="https://cloud.langfuse.com"
)

### Create the multi LLM langchain chain

In [14]:
from langchain.schema.output_parser import StrOutputParser

In [15]:
# chain = CONFIGURABLE_CHAT_PROMPT | bedrock_llm | StrOutputParser()
chain = (
    CONFIGURABLE_CHAT_PROMPT | bedrock_llm | StrOutputParser()
).with_config(RunnableConfig(callbacks=langfuse_handler))

In [16]:
chain \
    .with_config(configurable={"model": "claude_3_sonnet"}) \
    .invoke("What is a large language model ?")

AttributeError: 'LangchainCallbackHandler' object has no attribute 'parent_run_id'